In [1]:
import pandas
import numpy as np

from sklearn.preprocessing import OneHotEncoder

%matplotlib inline

In [2]:
def log_progress(sequence, every=10):
    from ipywidgets import IntProgress
    from IPython.display import display

    progress = IntProgress(min=0, max=len(sequence), value=0)
    display(progress)
    
    for index, record in enumerate(sequence):
        if index % every == 0:
            progress.value = index
        yield record

# Препроцессинг

Изначальный датасет очень большой и некоторые операции не возможны на моем ноутбуке в связи с нехваткой оперативной памяти, поэтому поделим датасет на `n` маленьких

In [4]:
data = pandas.read_csv("train.csv")
data.head()

,id3,user_id,id2,date,id1
0,714,464300,34,1,4
1,714,915655,34,1,4
2,316,262696,42,1,2
3,52,354280,4,1,10
4,581,218912,14,1,10


In [4]:
n_batches = 5
users = data["user_id"].unique()
users_batches = (users[i::n_batches] for i in range(n_batches))

for i, batch in enumerate(users_batches):
    i+=1
    print("batch %s from %s" % (i, n_batches))
    data[data["user_id"].isin(batch)].to_csv("data/raw/data_%s.csv" % i, index=False)

batch 1 from 5
batch 2 from 5
batch 3 from 5
batch 4 from 5
batch 5 from 5


# Пейплайн предобработки датасета

### Подготовка и обучение пейплайна

Считаем один блок данных, и протестируем на нем пейплайн.

In [38]:
dataset_name = "data/raw/data_1.csv"
data = pandas.read_csv(dataset_name)
data.head()

,id3,user_id,id2,date,id1
0,714,464300,34,1,4
1,590,1029729,63,1,9
2,703,861305,16,1,9
3,803,733016,40,1,7
4,219,448027,1,1,6


### OneHotEncoder
Закодируем все просмотры пользователя в некоторый вектор интересов, для этого обучим OHE на категориях и подкатегориях товаров.

In [70]:
ohe = OneHotEncoder()

ohe.fit(data[["id1","id2"]])

id1_columns = ["id1_%s" % i for i in range(data["id1"].unique().shape[0])]
id2_columns = ["id2_%s" % i for i in range(data["id2"].unique().shape[0])] 

x_column_names = ["user_id"] + id1_columns + id2_columns
column_names = x_column_names

### Векторизация просмотров пользователя

Это функция для получения веткора интересов пользователя.
По своей сути вектор отображает вероятность, что человек заинтересуется определенной темой, а так же вероятность того, что он заинтересуется подтемой определенной темы

In [161]:
def vectorize_user(log):
    # Преобразуем все id в one hot
    values = ohe.transform(log[["id1", "id2"]])
    
    #суммируем все вектора и получаем вектор истории просмотров
    values = np.array(values.sum(axis=0))[0].astype("int")
    
    user_id = np.array([data["user_id"].get_values()[0]])
    vector = np.concatenate([user_id, values])
    
    user_vector = pandas.DataFrame(columns=x_column_names, data=[vector])
    
    # нормализация вектора, для пролучения вероятностей
    for columns in [id1_columns, id2_columns]:
        vec_len = user_vector[columns].sum(axis=1)[0]
        
        for c in columns:
            user_vector[c] = user_vector[c]/vec_len
        
    return user_vector
    

Пример векторизации интересов пользователя:

In [163]:
data_ = data[data["user_id"] == 464300]
vectorize_user(data_)

,user_id,id1_0,id1_1,id1_2,id1_3,id1_4,id1_5,id1_6,id1_7,id1_8,...,id2_112,id2_113,id2_114,id2_115,id2_116,id2_117,id2_118,id2_119,id2_120,id2_121
0,464300,0.0,0.010563,0.193662,0.035211,0.105634,0.003521,0.021127,0.03169,0.267606,...,0.0,0.0,0.0,0.0,0.007042,0.024648,0.003521,0.0,0.056338,0.0


### Получение target значений

В качестве таргет значений мы будем предсказывать товары которые купит человек на следующей неделе, основываясь просмотрах товаров в прошлом.

Эта функция позволяет разделить данные на два блока: блок для векторизации истории, блок для получения таргет значений.

In [173]:
def split_x_y(log):
    if log["date"].unique().size<7: 
        return None
    
    split_date = data_["date"].max()-7
    
    return log[log["date"]>split_date], log[log["date"]<split_date]